In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
xgb.__version__

/home/tunguz/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.6.0rc1'

In [3]:
!nvidia-smi

Thu Apr 21 13:17:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   32C    P0    61W / 275W |    902MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:47:00.0 Off |                    0 |
| N/A   

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM-80GB (UUID: GPU-a19c00c3-2832-fe38-1c43-c18db3e909da)
GPU 1: NVIDIA A100-SXM-80GB (UUID: GPU-58b97c92-e879-49d3-85b5-1d9615f10873)
GPU 2: NVIDIA A100-SXM-80GB (UUID: GPU-d21cfed4-2e1a-f313-839c-ea008aca027a)
GPU 3: NVIDIA DGX Display (UUID: GPU-bda05ff2-94b1-c885-24f7-586c8f86b408)
GPU 4: NVIDIA A100-SXM-80GB (UUID: GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac)


In [5]:
train_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-feb-2021/xgtrain.csv')
test_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-feb-2021/xgval.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=["GPU-a19c00c3-2832-fe38-1c43-c18db3e909da",
                                                 "GPU-58b97c92-e879-49d3-85b5-1d9615f10873",
                                                 "GPU-d21cfed4-2e1a-f313-839c-ea008aca027a",
                                                 "GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac"])
client = Client(cluster)

/home/tunguz/.local/lib/python3.8/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37323 instead
  warnings.warn(
2022-04-21 13:18:51,564 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-21 13:18:51,581 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-21 13:18:51,592 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-21 13:18:51,600 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [8]:
!nvidia-smi

Thu Apr 21 13:19:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   32C    P0    61W / 275W |   1313MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:47:00.0 Off |                    0 |
| N/A   

In [10]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [11]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)

In [12]:
num_round = 1000

In [13]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['reg:squarederror']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['rmse']),

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    rmse = mean_squared_error(test_y, predictions, squared=False)
    
    return rmse

In [15]:
study = optuna.create_study(direction='minimize')

[I 2022-04-21 13:21:09,568] A new study created in memory with name: no-name-83eba753-be77-4219-a269-1c30bd60bfff


In [16]:
%%time
study.optimize(objective, n_trials=3)

[13:21:12] task [xgboost.dask]:tcp://127.0.0.1:33337 got new rank 0
[I 2022-04-21 13:28:07,419] Trial 0 finished with value: 1.4327292355984982 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'lambda': 0.03306175742611288, 'alpha': 0.012759958256318512, 'colsample_bytree': 0.5158250646051545, 'subsample': 0.847927979275615, 'learning_rate': 0.0017977213288108476, 'max_depth': 17, 'min_child_weight': 17, 'eval_metric': 'rmse'}. Best is trial 0 with value: 1.4327292355984982.
[13:28:07] task [xgboost.dask]:tcp://127.0.0.1:33337 got new rank 0
[I 2022-04-21 13:28:34,942] Trial 1 finished with value: 0.8498183733674679 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'lambda': 0.004477114632230474, 'alpha': 0.04305502109718364, 'colsample_bytree': 0.7391527216829525, 'subsample': 0.8824515175345107, 'learning_rate': 0.040390475706009066, 'max_depth': 13, 'min_child_weight': 215, 'eval_metric': 'rmse'}. Best is trial 1 with value: 0.8

CPU times: user 12.9 s, sys: 13.6 s, total: 26.5 s
Wall time: 7min 51s


In [17]:
%%time
study.optimize(objective, n_trials=20)

[13:29:32] task [xgboost.dask]:tcp://127.0.0.1:33337 got new rank 0
[I 2022-04-21 13:29:57,935] Trial 3 finished with value: 0.844085652162971 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'lambda': 0.1629969560656772, 'alpha': 0.7763089847397154, 'colsample_bytree': 0.7871249149275015, 'subsample': 0.795373381278987, 'learning_rate': 0.008504345418890261, 'max_depth': 11, 'min_child_weight': 184, 'eval_metric': 'rmse'}. Best is trial 3 with value: 0.844085652162971.
[13:29:57] task [xgboost.dask]:tcp://127.0.0.1:33337 got new rank 0
[I 2022-04-21 13:30:21,512] Trial 4 finished with value: 0.8506177477444197 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'lambda': 0.0048860548444564665, 'alpha': 6.837906682871727, 'colsample_bytree': 0.42613953313153535, 'subsample': 0.6468450080624281, 'learning_rate': 0.052829294261213855, 'max_depth': 11, 'min_child_weight': 154, 'eval_metric': 'rmse'}. Best is trial 3 with value: 0.844085

CPU times: user 20.5 s, sys: 15.9 s, total: 36.4 s
Wall time: 11min 54s


/home/tunguz/.local/lib/python3.8/site-packages/dask_cuda/initialize.py:57: UserWarning: Worker with process ID 3032142 should have a CUDA context assigned to device GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac, but instead the CUDA context is on device 4. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/dask_cuda/initialize.py:57: UserWarning: Worker with process ID 3032147 should have a CUDA context assigned to device GPU-58b97c92-e879-49d3-85b5-1d9615f10873, but instead the CUDA context is on device 1. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warning